In [ ]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO
import math

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture (input your video here)
cap = cv2.VideoCapture("/content/videoplayback.mp4")

# Line coordinates for counting
START_LINE = sv.Point(182, 254)
END_LINE = sv.Point(462, 254)

# Lane boundaries for lane detection (adjust as per your road setup)
LEFT_LANE_START = sv.Point(100, 254)
LEFT_LANE_END = sv.Point(300, 254)

RIGHT_LANE_START = sv.Point(320, 254)
RIGHT_LANE_END = sv.Point(520, 254)

# Dictionary to store vehicle history for tracking
track_history = defaultdict(lambda: [])
# Dictionary to track objects that have crossed the line
crossed_objects = {}
# Dictionary for lane changes
lane_changes = defaultdict(int)
# Dictionary for vehicle speeds
vehicle_speeds = {}
# Vehicle counts based on type (class)
vehicle_counts = defaultdict(int)

# Function to calculate speed (assuming frame rate and real-world scale)
def calculate_speed(x1, y1, x2, y2, fps, distance_scale):
    pixel_distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    speed = (pixel_distance / distance_scale) * fps * 3.6  # km/h
    return speed

# Open a video sink for output video
video_info = sv.VideoInfo.from_video_path("/content/videoplayback.mp4")
with sv.VideoSink("output_with_extra_features.mp4", video_info) as sink:

    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get FPS from the video
    distance_scale = 50  # Scale from pixels to real-world distance, adjust this

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the bounding boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id  # Check if track_ids exist

            # Only proceed if track_ids is not None
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()

                # Visualize the results on the frame
                annotated_frame = results[0].plot()
                detections = sv.Detections.from_ultralytics(results[0])

                # Process each tracked object
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    current_center = (float(x), float(y))
                    track.append(current_center)

                    # Keep only the last 30 tracks for smoothing
                    if len(track) > 30:
                        track.pop(0)

                    # Speed estimation: Check if there is enough history to calculate speed
                    if len(track) > 1:
                        prev_center = track[-2]
                        speed = calculate_speed(prev_center[0], prev_center[1], current_center[0], current_center[1], fps, distance_scale)
                        vehicle_speeds[track_id] = speed

                    # Check if the object crosses the counting line
                    if START_LINE.x < x < END_LINE.x and abs(y - START_LINE.y) < 5:
                        if track_id not in crossed_objects:
                            crossed_objects[track_id] = True
                            vehicle_type = results[0].names[results[0].boxes.cls.int().cpu().tolist()[0]]
                            vehicle_counts[vehicle_type] += 1
                            print(f"Vehicle {track_id} ({vehicle_type}) crossed the line")

                        # Color the box based on the vehicle type
                        color = (0, 255, 0) if vehicle_type == 'car' else (0, 0, 255) if vehicle_type == 'bus' else (255, 255, 0)
                        cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), color, 2)

                    # Lane change detection
                    if LEFT_LANE_START.x < x < RIGHT_LANE_END.x:
                        if x < LEFT_LANE_END.x and lane_changes[track_id] == 0:
                            lane_changes[track_id] = 1  # Left lane
                            print(f"Vehicle {track_id} entered Left Lane")
                        elif x > RIGHT_LANE_START.x and lane_changes[track_id] == 1:
                            lane_changes[track_id] = 2  # Right lane (lane change detected)
                            print(f"Vehicle {track_id} changed to Right Lane")
                            cv2.putText(annotated_frame, f"Lane Change Detected: {track_id}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                    # Speed alert if the vehicle exceeds the speed limit (e.g., 80 km/h)
                    if track_id in vehicle_speeds and vehicle_speeds[track_id] > 80:
                        cv2.putText(annotated_frame, f"Speeding: {int(vehicle_speeds[track_id])} km/h", (int(x), int(y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                    # Draw the counting line and lanes on the frame
                    cv2.line(annotated_frame, (START_LINE.x, START_LINE.y), (END_LINE.x, END_LINE.y), (0, 255, 0), 2)
                    cv2.line(annotated_frame, (LEFT_LANE_START.x, LEFT_LANE_START.y), (LEFT_LANE_END.x, LEFT_LANE_END.y), (255, 0, 0), 2)
                    cv2.line(annotated_frame, (RIGHT_LANE_START.x, RIGHT_LANE_START.y), (RIGHT_LANE_END.x, RIGHT_LANE_END.y), (255, 0, 0), 2)

                # Display the count of different vehicle types on the frame
                count_text = f"Cars: {vehicle_counts['car']} Buses: {vehicle_counts['bus']} Trucks: {vehicle_counts['truck']}"
                cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # Write the annotated frame to the output video
                sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 2 cars, 1 truck, 168.6ms
Speed: 5.9ms preprocess, 168.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track2

0: 384x640 2 cars, 1 truck, 182.6ms
Speed: 5.9ms preprocess, 182.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track2

0: 384x640 1 car, 1 bus, 2 trucks, 153.2ms
Speed: 3.8ms preprocess, 153.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track2

0: 384x640 2 cars, 2 trucks, 153.5ms
Speed: 3.8ms preprocess, 153.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track2

0: 384x640 2 cars, 1 truck, 174.1ms
Speed: 6.1ms preprocess, 174.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track2

0: 384x640 1 car, 2 trucks, 155.1ms
Speed: 4.7ms preprocess, 155.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Re

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 13.4 MB/s eta 0:00:00
